In [1]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
from mpltools import special
from qiskit import *
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)
from qiskit.quantum_info.synthesis import OneQubitEulerDecomposer
from qiskit.tools.visualization import plot_histogram
from qiskit.extensions import Initialize
from qiskit.tools.monitor import job_monitor
IBMQ.save_account('0a1c53ad870ce01b732d806b4767e84fbdafb8d3278f97443e696c837282b2095361917ce1ed97757fcc71816dffa996363b64f7d4e27263a1947c5ccafe2ac0', overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')
import pandas as pd
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeQuito

In [3]:
cols = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8',
        'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16',
        'II_00', 'II_01', 'II_10', 'II_11', 'IX_00', 'IX_01', 'IX_10', 'IX_11',
        'IY_00', 'IY_01', 'IY_10', 'IY_11', 'IZ_00', 'IZ_01', 'IZ_10', 'IZ_11',
        'XI_00', 'XI_01', 'XI_10', 'XI_11', 'XX_00', 'XX_01', 'XX_10', 'XX_11',
        'XY_00', 'XY_01', 'XY_10', 'XY_11', 'XZ_00', 'XZ_01', 'XZ_10', 'XZ_11',
        'YI_00', 'YI_01', 'YI_10', 'YI_11', 'YX_00', 'YX_01', 'YX_10', 'YX_11',
        'YY_00', 'YY_01', 'YY_10', 'YY_11', 'YZ_00', 'YZ_01', 'YZ_10', 'YZ_11',
        'ZI_00', 'ZI_01', 'ZI_10', 'ZI_11', 'ZX_00', 'ZX_01', 'ZX_10', 'ZX_11',
        'ZY_00', 'ZY_01', 'ZY_10', 'ZY_11', 'ZZ_00', 'ZZ_01', 'ZZ_10', 'ZZ_11']

In [7]:
#Create #datapoints and save dataframe corresponding to cols

qbts = 2
shots = 1000
data_pts = 1000
device_backend = FakeQuito()
sim_quito = AerSimulator.from_backend(device_backend)

lk = 0
for jk in range(data_pts):
    desired_vector = np.array(qt.rand_ket_haar(2**qbts)).reshape(2**qbts)

    iter_arr = np.array(qt.ket2dm(qt.Qobj(desired_vector))).reshape(-1)

    for ij in range(4**qbts):
        
        qr = QuantumRegister(qbts)
        cr = ClassicalRegister(qbts)
        circ = QuantumCircuit(qr, cr)
        circ.initialize(desired_vector, [qr[0],qr[1]])
        
        if ij == 0:
            #II measurement
            circ.measure(qr, cr)

        if ij == 1:
            #IX measurement
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 2:
            #IY measurement
            circ.sdg(qr[1])
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 3:
            #IZ measurement
            circ.measure(qr, cr)

        if ij == 4:
            #XI measurement
            circ.h(qr[0])
            circ.measure(qr, cr)

        if ij == 5:
            #XX measurement
            circ.h(qr[0])
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 6:
            #XY measurement
            circ.h(qr[0])
            circ.sdg(qr[1])
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 7:
            #XZ measurement
            circ.h(qr[0])
            circ.measure(qr, cr)

        if ij == 8:
            #YI measurement
            circ.sdg(qr[0])
            circ.h(qr[0])
            circ.measure(qr, cr)

        if ij == 9:
            #YX measurement
            circ.sdg(qr[0])
            circ.h(qr[0])
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 10:
            #YY measurement
            circ.sdg(qr[0])
            circ.h(qr[0])
            circ.sdg(qr[1])
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 11:
            #YZ measurement
            circ.sdg(qr[0])
            circ.h(qr[0])
            circ.measure(qr, cr)

        if ij == 12:
            #ZI measurement
            circ.measure(qr, cr)

        if ij == 13:
            #ZX measurement
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 14:
            #ZY measurement
            circ.sdg(qr[1])
            circ.h(qr[1])
            circ.measure(qr, cr)

        if ij == 15:
            #ZZ measurement
            circ.measure(qr, cr)

        # circuit.draw(output='mpl')

        # Transpile the circuit for the noisy basis gates
        tcirc = transpile(circ, sim_quito)

        # Execute noisy simulation and get counts
        result_noise = sim_quito.run(tcirc, shots = shots).result()
        counts_noise = result_noise.get_counts(0)
        
        data = {
            '00': counts_noise['00'],
            '01': counts_noise['01'],
            '10': counts_noise['10'],
            '11': counts_noise['11']
               }
        
        temp_df = pd.DataFrame(data, index = [0])
        iter_arr = np.append(iter_arr, temp_df.loc[0].to_numpy())
    
    if len(iter_arr) == 80:
        df_2 = pd.DataFrame(iter_arr.reshape(1, 80), columns = cols, index=[jk])
        df_2.to_csv('Pure_4d_Data.csv', mode = 'a', header = jk==0)
    
    if jk % 100 == 0:
        print("Data Points Saved = {}e02".format(lk))
        lk += 1

# plot_histogram(counts_noise,
#title="Counts for 2-qubit pure state with IBM_Quito noise model")

Data Points Saved = 0e02
Data Points Saved = 1e02
Data Points Saved = 2e02
Data Points Saved = 3e02
Data Points Saved = 4e02
Data Points Saved = 5e02
Data Points Saved = 6e02
Data Points Saved = 7e02
Data Points Saved = 8e02
Data Points Saved = 9e02


In [7]:
shots = 1000

I = np.array([[1, 0],
              [0, 1]], dtype = 'complex')
X = np.array([[0, 1],
              [1, 0]], dtype = 'complex')
Y = np.array([[0, -1j],
              [1j, 0]], dtype = 'complex')
Z = np.array([[1, 0],
              [0, -1]], dtype = 'complex')

pauli = np.array([I, X, Y ,Z], dtype = 'complex')
mn_ideal = np.zeros([16, 4], dtype = 'complex')
df_2_r = df_2.iloc[0,16:].to_numpy(dtype = 'complex').reshape(16,4)
# mn_rnd = np.zeros([16, 4])

for ij in range(4):
    for jk in range(4):
        optr = np.kron(pauli[ij], pauli[jk])
        eig_val, eig_vec = np.linalg.eig(optr)
        for kl in range(4):
            proj = np.matrix(eig_vec[:, kl]).T
            mn_ideal[4*ij + jk, kl] = np.trace(np.matmul(rho, np.matmul(proj, proj.H)))*eig_val[kl]
            df_2_r[4*ij + jk, kl] = df_2_r[4*ij + jk, kl]*eig_val[kl]

mn_ideal = (1000*mn_ideal).round()
mn_ideal

array([[ 320.+0.j,  411.+0.j,   98.+0.j,  171.+0.j],
       [ 406.+0.j, -325.+0.j,   93.+0.j, -176.+0.j],
       [ 726.+0.j,   -5.+0.j,  257.+0.j,  -12.+0.j],
       [ 320.+0.j, -411.+0.j,   98.+0.j, -171.+0.j],
       [ 367.+0.j,  -51.+0.j,  556.+0.j,  -26.+0.j],
       [ 279.+0.j, -213.+0.j,  184.+0.j, -325.+0.j],
       [ 477.+0.j,  -14.+0.j,  442.+0.j,  -67.+0.j],
       [ 367.+0.j,  -51.+0.j,   26.+0.j, -556.+0.j],
       [ 129.+0.j, -289.+0.j,  283.+0.j, -299.+0.j],
       [ 477.+0.j,  -14.+0.j,   67.+0.j, -442.+0.j],
       [ 213.+0.j, -279.+0.j,  184.+0.j, -325.+0.j],
       [ 129.+0.j, -289.+0.j,  299.+0.j, -283.+0.j],
       [ 320.+0.j,  411.+0.j,  -98.+0.j, -171.+0.j],
       [ 406.+0.j, -325.+0.j,  176.+0.j,  -93.+0.j],
       [ 726.+0.j,   -5.+0.j,   12.+0.j, -257.+0.j],
       [ 320.+0.j, -411.+0.j,  -98.+0.j,  171.+0.j]])

In [8]:
r = np.sum(mn_ideal, axis = 1)/1000
r = r.reshape(4,4)
r = r/4
df_2_r = np.sum(df_2_r, axis=1)/4000
df_2_r = df_2_r.reshape(4,4)

state = np.zeros([4,4], dtype = 'complex')
state_fq = np.zeros([4,4], dtype = 'complex')

for ij in range(4):
    for jk in range(4):
        state += r[ij, jk]*np.kron(pauli[ij], pauli[jk])
        state_fq += df_2_r[ij, jk]*np.kron(pauli[ij], pauli[jk])
        
display(qt.Qobj(state), qt.Qobj(state_fq), qt.Qobj(rho))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.32  +0.j      0.0405-0.3605j  0.158 +0.08j    0.033 -0.2315j]
 [ 0.0405+0.3605j  0.411 +0.j     -0.0705+0.1875j  0.265 +0.008j ]
 [ 0.158 -0.08j   -0.0705-0.1875j  0.098 +0.j     -0.0415-0.1225j]
 [ 0.033 +0.2315j  0.265 -0.008j  -0.0415+0.1225j  0.171 +0.j    ]]

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.404+0.j      0.058-0.0285j  0.006-0.023j   0.05 -0.2175j]
 [ 0.058+0.0285j  0.323+0.j     -0.044-0.1705j  0.018+0.22j  ]
 [ 0.006+0.023j  -0.044+0.1705j  0.16 +0.j     -0.019-0.0675j]
 [ 0.05 +0.2175j  0.018-0.22j   -0.019+0.0675j  0.113+0.j    ]]

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.32024568+0.j          0.04025425-0.36042672j  0.15791412+0.08011981j
   0.03301339-0.23176415j]
 [ 0.04025425+0.36042672j  0.41070913+0.j         -0.07032291+0.18779841j
   0.26499319+0.00802327j]
 [ 0.15791412-0.08011981j -0.07032291-0.18779841j  0.0979125 +0.j
  -0.0417043 -0.12254298j]
 [ 0.03301339+0.23176415j  0.26499319-0.00802327j -0.0417043 +0.12254298j
   0.1711327 +0.j        ]]

In [208]:
mn_ideal

array([[ 264.+0.j,  239.+0.j,  196.+0.j,  302.+0.j],
       [ 107.+0.j, -396.+0.j,  207.+0.j, -290.+0.j],
       [ 457.+0.j,  -46.+0.j,    9.+0.j, -488.+0.j],
       [ 264.+0.j, -239.+0.j,  196.+0.j, -302.+0.j],
       [  12.+0.j, -447.+0.j,  399.+0.j, -142.+0.j],
       [ 408.+0.j, -158.+0.j,  387.+0.j,  -48.+0.j],
       [ 536.+0.j,  -30.+0.j,   83.+0.j, -352.+0.j],
       [  12.+0.j, -447.+0.j,  142.+0.j, -399.+0.j],
       [ 294.+0.j, -165.+0.j,   35.+0.j, -506.+0.j],
       [ 536.+0.j,  -30.+0.j,  352.+0.j,  -83.+0.j],
       [ 158.+0.j, -408.+0.j,  387.+0.j,  -48.+0.j],
       [ 294.+0.j, -165.+0.j,  506.+0.j,  -35.+0.j],
       [ 264.+0.j,  239.+0.j, -196.+0.j, -302.+0.j],
       [ 107.+0.j, -396.+0.j,  290.+0.j, -207.+0.j],
       [ 457.+0.j,  -46.+0.j,  488.+0.j,   -9.+0.j],
       [ 264.+0.j, -239.+0.j, -196.+0.j,  302.+0.j]])

In [48]:
temp3 = pd.DataFrame(np.tile(np.array([counts_noise['00'], counts_noise['01'], counts_noise['10'],counts_noise['11']]), [10,1]).reshape(10,4),
             columns = ['00', '01', '10', '11'])

In [53]:
temp3['00'].to_numpy().reshape(10,1) + temp3['11'].to_numpy().reshape(10,1)

array([[770],
       [770],
       [770],
       [770],
       [770],
       [770],
       [770],
       [770],
       [770],
       [770]])